# 🧠 NLP Topic Classification Model (Stage B)

This notebook handles the **Training Phase** of the pipeline:
1.  **Load Data**: Reads the clean `nlp_dataset.jsonl` created in Stage A.
2.  **Tokenization**: Segments Vietnamese words using `pyvi` (e.g., "Hà Nội" -> "Hà_Nội").
3.  **Vectorization**: Converts text to numbers using **TF-IDF** (removing stopwords here).
4.  **Modeling**: Trains a **Logistic Regression** classifier.
5.  **Evaluation**: Checks Accuracy, F1-Score, and analyzes specific Error cases.
6.  **Export**: Saves the trained model for deployment.

In [1]:
# --- 1. SETUP & IMPORTS ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from pyvi import ViTokenizer

# Scikit-learn modules
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

# Configuration
pd.set_option('display.max_colwidth', 200)
DATA_FILE = "nlp_dataset.jsonl"
MODEL_PATH = "nlp_topic_classifier.pkl"

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 2. Load Data & Prepare Stopwords

In [2]:
# Load Dataset
print(f"📂 Loading dataset from {DATA_FILE}...")

try:
    df = pd.read_json(DATA_FILE, lines=True)
    print(f"✅ Loaded {len(df)} records.")
except ValueError:
    print("❌ Error: Dataset file not found. Please run 'create_dataset.py' first.")

# Define Vietnamese Stopwords
# These words are grammatical glue that don't carry much topic meaning
STOPWORDS = {
    "của", "và", "các", "có", "được", "cho", "trong", "một", "là", "với", "để", 
    "những", "khi", "này", "khi", "tại", "đã", "thì", "mà", "như", "năm", "từ",
    "đến", "người", "sẽ", "cũng", "về", "vào", "ra", "nên", "nếu", "bị", "bởi",
    "lại", "do", "nhưng", "đang", "vẫn", "chỉ", "theo", "gì", "ai", "đâu", "rằng"
}
STOPWORDS_LIST = list(STOPWORDS)

# Preview data
df[['label_name', 'text']].head(3)

📂 Loading dataset from nlp_dataset.jsonl...
✅ Loaded 126687 records.


,label_name,text
0,Ẩm thực,"Một trong những phân khu sôi động nhất Hội chợ Mùa Thu 2025 đang diễn ra tại Trung tâm Hội chợ Triển lãm Quốc gia (ở xã Đông Anh, Hà Nội) là “Thu mỹ vị”. Khu vực này đem đến một đại tiệc ẩm thực t..."
1,Ẩm thực,"Chủ đề về ẩm thực Hà thành luôn thu hút sự quan tâm của nhiều người. Sau khi Dân trí đăng tải bài viết Tranh cãi mâm cỗ nghệ nhân ở Bát Tràng 6 người giá 2,7 triệu đồng đắt đỏ, độc giả và bạn đọc ..."
2,Ẩm thực,"Tự làm bánh mỳ, bánh phở, nhập mắm tôm từ Việt Nam sang Mỹ\n\nTrong bài viết được tác giả Helen Rosner chia sẻ nhận định trên tờ New Yorker, một nhà hàng do người Việt Nam làm chủ ở khu Upper West..."


## 3. Vietnamese Word Segmentation (Tách từ)
Using `pyvi` to join compound words with underscores (e.g., `học sinh` -> `học_sinh`). This is crucial for TF-IDF to treat them as single tokens.

In [ ]:
print("⚙️ Tokenizing text (This may take a minute)...")

def segment_text(text):
    if not isinstance(text, str): return ""
    return ViTokenizer.tokenize(text)

# Apply tokenization to a new column
df['segmented_text'] = df['text'].apply(segment_text)

print("✅ Tokenization Complete!")
print("Sample Original:  ", df['text'].iloc[0][:60])
print("Sample Segmented: ", df['segmented_text'].iloc[0][:60])

## 4. Train - Test Split
We split the data **BEFORE** vectorization to prevent data leakage.

In [ ]:
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    df['segmented_text'], 
    df['label_id'], 
    df.index, # Keep index to trace back filenames later
    test_size=0.2, 
    random_state=42, 
    stratify=df['label_id'] # Ensure balanced classes in test set
)

print(f"🔹 Training Set: {len(X_train)} samples")
print(f"🔹 Test Set:     {len(X_test)} samples")

## 5. Build & Train Pipeline
We use a `Pipeline` to bundle:
1.  **TfidfVectorizer**: Converts text to vectors, filtering stopwords and rare words.
2.  **LogisticRegression**: A fast and effective baseline model for text classification.

In [ ]:
print("🚀 Training Model...")

model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,      # Keep top 20k important words
        ngram_range=(1, 2),      # Use unigrams and bigrams
        stop_words=STOPWORDS_LIST,
        min_df=2                 # Ignore words appearing < 2 times
    )),
    ('clf', LogisticRegression(
        solver='sag',            # Optimized for large datasets
        multi_class='multinomial', 
        max_iter=1000,
        n_jobs=-1
    ))
])

model_pipeline.fit(X_train, y_train)

print("✅ Training Finished.")

## 6. Evaluation
Assessing model performance on the Test set.

In [ ]:
# Predictions
y_pred = model_pipeline.predict(X_test)

# Get correct label names map
label_map = df.drop_duplicates('label_id').sort_values('label_id')
label_names = label_map['label_name'].tolist()

# Metrics
acc = accuracy_score(y_test, y_pred)
print(f"🏆 Accuracy: {acc:.2%}\n")

print("📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_names))

### Confusion Matrix Visualization

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_names, 
            yticklabels=label_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

## 7. Error Analysis (Quan trọng)
Identify which specific files confused the model. This is possible because we preserved the `filename`.

In [ ]:
# Reconstruct Test DataFrame with predictions
test_results = df.loc[idx_test].copy()
test_results['predicted_id'] = y_pred
test_results['predicted_label'] = test_results['predicted_id'].apply(lambda x: label_names[x])

# Filter Errors
errors = test_results[test_results['label_id'] != test_results['predicted_id']]

print(f"⚠️ Total Misclassified: {len(errors)} / {len(X_test)}")
print("samples of errors:")

# Show random 5 errors
if not errors.empty:
    cols_to_show = ['filename', 'label_name', 'predicted_label', 'text']
    display(errors[cols_to_show].sample(min(5, len(errors))))
else:
    print("Wow! No errors found. Perfect model? Check for overfitting.")

## 8. Save Model & Live Inference

In [ ]:
# Save pipeline
joblib.dump(model_pipeline, MODEL_PATH)
print(f"💾 Model saved to: {MODEL_PATH}")

# --- Inference Function ---
def predict_news(text):
    # Preprocessing (must match training)
    tokenized = ViTokenizer.tokenize(text)
    
    # Predict
    pred_id = model_pipeline.predict([tokenized])[0]
    prob = model_pipeline.predict_proba([tokenized]).max()
    
    return label_names[pred_id], prob

# Try it out
sample_news = "Giá chung cư tại Hà Nội tiếp tục lập đỉnh mới do nguồn cung khan hiếm."
topic, conf = predict_news(sample_news)

print(f"\n📰 Input: {sample_news}")
print(f"🤖 Prediction: {topic} (Confidence: {conf:.1%})")